### Format and send data to TimescaleDB


In [54]:
import os
import requests
import pandas as pd
import json
from datetime import datetime

### OCRing the receipt


In [14]:
receiptOcrEndpoint = "https://ocr.asprise.com/api/v1/receipt" # Receipt OCR API endpoint
folder_path_img = "receipts/jpg" #folder path
folder_path_json = "receipts/json" 
folder_path_pdf = "receipts/pdf"

### 1) Convert pdf to jpg


In [4]:
from pdf2image import convert_from_path
 
for filename in os.listdir(folder_path_pdf):
    if filename.endswith(".pdf"):
        pdf_path = os.path.join(folder_path_pdf, filename)
        
        # Store Pdf with convert_from_path function
        images = convert_from_path(pdf_path)
        
        for i in range(len(images)): 
            # Save pages as images in the pdf
            jpg_filename = folder_path_img + "/" + os.path.splitext(filename)[0] +".jpg"
            images[i].save(jpg_filename, "JPEG")

        print(f"Processed {filename} - Result saved to {jpg_filename}")


### 2) Read jpg file and save into json


In [6]:
OverAsked = True

In [31]:
#iterate over the files in the folder 
if OverAsked == False:
  for filename in os.listdir(folder_path_img):
    if filename.endswith(".jpg"):
        image_path = os.path.join(folder_path_img, filename)
        r = requests.post(receiptOcrEndpoint, data = {
        "client_id": "TEST",        # Use "TEST' for testing purpose /
        'recognizer': 'auto',       # can be 'US', "CA', 'JP', 'SG' or 'auto' /
        'ref_no': 'ocr_python_123', # optional caller provided ref code /
        },
        files = {"file": open(image_path, "rb")})

        # Save the json response to a file
        json_filename = os.path.splitext(filename)[0] + ".json"
        json_path = os.path.join(folder_path_json, json_filename)        
        with open(json_path, "w") as outfile:
            json.dump(r.json(), outfile)
            #https://www.geeksforgeeks.org/reading-and-writing-json-to-a-file-in-python/

        print(f"Processed {filename} - Result saved to {json_filename}")
        print(r.text)

## 3) All in one


In [60]:
#init translator
from googletrans import Translator
translator = Translator()
translation_flag = False
#initialize table
products = []
items = []
# init parameters
merchant_name = []
merchant_address = []
country = []
date = []
total = []
taxes = []


for filename in os.listdir(folder_path_json):
    #read json file
    if filename.endswith(".json"):
        json_path = os.path.join(folder_path_json, filename)
        #Read all json in json folder
        with open(json_path, "r") as openfile:
            #reading from json file
            json_object = json.load(openfile)
        print(filename, ": ", json_object)

        if json_object["success"] != False:
            #Sort the items, ... in the reicept
            if json_object["receipts"] is not None:
                receipt_object = json_object["receipts"]
                #receipt_object.append(json_object["receipts"])
                # Access the "items" list within the "receipts" object
                items = json_object["receipts"][0]["items"]
                #items.append(json_object["receipts"][0]["items"])
                #print("items: ",items)
                total.append(json_object["receipts"][0]["total"])
                # print("total: ",total)
                taxes.append(json_object["receipts"][0]["tax"])
                merchant_name.append(json_object["receipts"][0]["merchant_name"])
                merchant_address.append(json_object["receipts"][0]["merchant_address"])
                country.append(json_object["receipts"][0]["country"])
                date_str = str(json_object["receipts"][0]["date"]) if json_object["receipts"][0]["date"] else str(datetime.now().strftime("%Y-%m-%d"))
                date_str += " "
                date_str += str(json_object["receipts"][0]["time"]) if json_object["receipts"][0]["time"] else str("19:00")
                date.append(date_str)
                print("Grocery named: %s, %s" % (json_object["receipts"][0]["merchant_name"], date_str))

                #add for each item merchant_name country, date
                if items is not None:
                    for item in items:
                        item.update({"merchant_name": json_object["receipts"][0]["merchant_name"]})
                        item.update({"merchant_address": json_object["receipts"][0]["merchant_address"]})
                        item.update({"country": json_object["receipts"][0]["country"]})
                        item.update({"date": date_str})
                else:
                    print("Any items detected")  
            else:
                print("Error is"+json_object["message"])
        if items is not None:
            #translate all the descriptions
            try:
                if translation_flag:
                    for item in items: 
                        translation = translator.translate(item["description"], dest="fr")
                        item["description"] = translation.text
                        #print("Description : ", item["description"])
                print(f"Translation mode : {translation_flag}")
            except:
                print("translation canceled")   

            #find all the product, only the product -> find with tag ?
            for item in items:
                flag = item["flags"]
                if flag != "":
                    products.append(item)
            #print(products)
        else:
            print("Any items detected -- check OCRing method")
        
#save data into csv file
if items and products is not None:
    #export product all
    path_csv = 'receipts/export_products/'
    formatted_datetime = datetime.now().strftime("%Y-%m-%dT%Hh%Mmin%Ss")
    # Create the new file name with the formatted datetime
    file_name_csv = f"{path_csv}"
    if translation_flag:
        file_name_csv += f"export_FR/export_products_FR_{formatted_datetime}.csv"
        pd.DataFrame(pd.DataFrame(products)[["date", "description", "amount", "merchant_name", "merchant_address", "country", "flags"]]).to_csv(file_name_csv, index=False, sep=";")
    else:
        file_name_csv += f"export_DE/export_products_DE_{formatted_datetime}.csv"
        pd.DataFrame(pd.DataFrame(products)[["date", "description", "amount", "merchant_name", "merchant_address", "country", "flags"]]).to_csv(file_name_csv, index=False, sep=";")


lidl-03-06-23-1.json :  {'ocr_type': 'receipts', 'request_id': 'P_92.184.117.81_lj1wnslk_cvi', 'ref_no': 'ocr_python_123', 'file_name': 'lidl-03-06-23.jpg', 'request_received_on': 1687121634441, 'success': True, 'image_width': 845, 'image_height': 2200, 'image_rotation': 0.014, 'recognition_completed_on': 1687121634987, 'receipts': [{'merchant_name': 'LIDL Müllerstraße 48-48a', 'merchant_address': '13349 Berlin-Wedding', 'merchant_phone': None, 'merchant_website': None, 'merchant_tax_reg_no': None, 'merchant_company_reg_no': None, 'region': None, 'mall': None, 'country': 'DE', 'receipt_no': 'DE813388858', 'date': '2023-06-03', 'time': '20:45', 'items': [{'amount': 1.01, 'category': None, 'description': 'TomateStrau. kg', 'flags': ' A', 'qty': None, 'remarks': None, 'tags': None, 'unitPrice': None}, {'amount': 0.95, 'category': None, 'description': 'Eisbergsalat', 'flags': ' A', 'qty': None, 'remarks': None, 'tags': None, 'unitPrice': None}, {'amount': 1.99, 'category': None, 'descripti

### 4) Send csv to database


In [50]:
import psycopg2
conn = None

CSV_DIR = "C:\\Users\\leray\\OneDrive\\Documents\\Data_Workspace\\receipt_OCR_project\\receipt-ocr\\python-receipt-ocr\\receipts\\"
SQl_DIR = "SQL\\"

In [30]:
try:
    conn = psycopg2.connect(host='localhost',
                            database='timescaledbforreceipt',
                            user='postgres',
                            password='pg123')
    print('database connected')

    # Get a list of all the CSV files in the directory
    listdir = os.listdir(CSV_DIR)
    file_names = []
    for i in range(len(listdir)):
        file_names.append(CSV_DIR+listdir[i])

    with conn.cursor() as cur:
        conn.commit()
        cur.close()

    for file_name in file_names:
        with open(file_name, 'r') as file_in:
            next(file_in)
            with conn.cursor() as cur:
                # headers
                cur.execute("Select * FROM receipts LIMIT 0")
                headers = [desc[0] for desc in cur.description]
                # Now upload the data as though it was a file
                cur.copy_from(file_in, "receipts",
                              columns=headers, sep=";", null="")
                # caractère "" qui doivent être mis en null https://docs.postgresql.fr/12/sql-copy.html
            conn.commit()
    print("finish")
    cur.close()


except (Exception, psycopg2.DatabaseError) as error:
    print(error)

finally:
    if conn is not None:
        conn.close()  # close the communication with the PostgreSQL by calling the close() method of the connection object

database connected
[WinError 267] Nom de répertoire non valide: 'C:\\Users\\leray\\OneDrive\\Documents\\Data_Workspace\\receipt_OCR_project\\receipt-ocr\\python-receipt-ocr\\receipts\\export_products.csv'


### 5) Retrived data from the timescaleDB

https://www.timescale.com/blog/speeding-up-data-analysis/

In [40]:
try:
    conn = psycopg2.connect(host='localhost',
                            database='timescaledbforreceipt',
                            user='postgres',
                            password='pg123')
    print('database connected')
    # Execute a query and fetch the results
    query_select_all = "SELECT * FROM receipts"
    df_products = pd.read_sql_query(query_select_all, conn)
    with open(SQl_DIR+'GroupBy.sql', 'r') as file:
        query_group_by = file.read()
    df_products_groupby = pd.read_sql_query(query_group_by, conn)

except (Exception, psycopg2.DatabaseError) as error:
    print(error)

finally:
    if conn is not None:
        conn.close()  # close the communication with the PostgreSQL by calling the close() method of the connection object

database connected


C:\Users\leray\AppData\Local\Temp\ipykernel_7160\1764840774.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_products = pd.read_sql_query(query_select_all, conn)
C:\Users\leray\AppData\Local\Temp\ipykernel_7160\1764840774.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_products_groupby = pd.read_sql_query(query_group_by, conn)


In [48]:
df_products_groupby.head()


,day,sum,description,merchant_address
0,2023-06-03 00:00:00+00:00,1.99,Légumes asiatiques,13349 Berlin-Wedding
1,2023-06-03 00:00:00+00:00,0.95,Salade d'iceberg,13349 Berlin-Wedding
2,2023-06-03 00:00:00+00:00,0.89,Sauce de pomme sugared,13349 Berlin-Wedding
3,2023-06-03 00:00:00+00:00,1.95,Hygienespray Univer,13349 Berlin-Wedding
4,2023-06-03 00:00:00+00:00,1.01,Tomatestrau.kg,13349 Berlin-Wedding


### format datasets

In [44]:
#weekday
df_products['weekday']=df_products['time'].dt.weekday
df_products['weekday_label']=df_products['weekday']
dict_weekday = {'Monday': 1, 'Tuesday': 2, 'Wednesday': 3, 'Thursday': 4, 'Friday': 5, 'Saturday': 6, 'Sunday': 7}

for weekday in df_products['weekday']:
    for key, items in dict_weekday.items():
        if items == weekday:
            df_products['weekday_label'][weekday] = dict_weekday[key]
            
print(df_products['weekday_label'])

0      5
1      1
2      2
3      3
4      4
      ..
119    1
120    1
121    1
122    1
123    1
Name: weekday_label, Length: 124, dtype: int64


C:\Users\leray\AppData\Local\Temp\ipykernel_7160\3726481207.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_products['weekday_label'][weekday] = dict_weekday[key]
